In [1]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!nvidia-smi

Thu Jul  2 14:05:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14.0

# 数据处理

In [ ]:
!pip install hanziconv
!pip install pyhanlp
from pyhanlp import *

下载 https://file.hankcs.com/bin/hanlp-1.7.7-release.zip 到 /usr/local/lib/python3.6/dist-packages/pyhanlp/static/hanlp-1.7.7-release.zip
100.00%, 1 MB, 698 KB/s, 还有 0 分  0 秒   
下载 https://file.hankcs.com/hanlp/data-for-1.7.5.zip 到 /usr/local/lib/python3.6/dist-packages/pyhanlp/static/data-for-1.7.7.zip
94.84%, 604 MB, 6579 KB/s, 还有 0 分  5 秒   

In [6]:
% cd /content/gdrive/My\ Drive/Colab\ Notebooks/nlp_task/fine_gained/mycode
! pwd
! ls

/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
data		 elmo_no_all_drop.py  output	       run_log.ipynb
data_not_aug	 elmo.py	      preprocess.sh    weight_drop_lstm.py
data_process.py  elmo_run.py	      __pycache__
dataset.py	 elmo_utils.py	      run_log_@.ipynb


In [ ]:
!bash preprocess.sh

In [ ]:
# # 未增强的数据
# !python data_process.py \
#     --data_file='../../data/trainingset/sentiment_analysis_trainingset.csv' \
#     --output_file=data_not_aug/train.json \
#     --vocab_file=data_not_aug/vocab.txt \
# 	--is_trian_file=True \
#     --vocab_size=50000

# processed -- 10000 --
# processed -- 20000 --
# processed -- 30000 --
# processed -- 40000 --
# processed -- 50000 --
# processed -- 60000 --
# processed -- 70000 --
# processed -- 80000 --
# processed -- 90000 --
# processed -- 100000 --
# Start to create vocab ...
# Created vocab file data_not_aug/vocab.txt with vocab size 50000


In [ ]:
# !python data_process.py \
#     --data_file='../fsauor2018/wordvec/sgns.sogou.word' \
#     --output_file=data_not_aug/embedding.txt \
#     --vocab_file=data_not_aug/vocab.txt \
#     --embedding=True

# !python data_process.py \
#     --data_file='../../data/validationset/sentiment_analysis_validationset.csv' \
#     --output_file=data_not_aug/validation.json

# !python data_process.py \
#     --data_file='../../data/testa/sentiment_analysis_testa.csv' \
#     --output_file=data_not_aug/testa.json

# processed -- 10000 --
# processed -- 10000 --


In [ ]:
# import json

# max_len = 0
# count = 0
# with open('data/clip_trian.json', 'w',encoding='utf-8') as f:
#     for line in open('data/train.json'):
#         item = json.loads(line.strip())
#         content = item['content']
#         tokens = content.strip().split()
#         length = len(tokens)
#         if length > max_len:
#             max_len = length
#         if length > 1000:
#             count += 1
#             tokens = tokens[: 500] +  tokens[-500: ]
#         item['content'] = ' '.join(tokens)
#         abc = json.dumps(item, ensure_ascii=False) + '\n'
#         f.write(abc)
        
# print(max_len)
# print(count)

1493
297


In [ ]:
# max_len = 0
# count = 0
# for line in open('data/clip_trian.json'):
#         item = json.loads(line.strip())
#         content = item['content']
#         tokens = content.strip().split()
#         length = len(tokens)
#         if length > max_len:
#             max_len = length
#         if length > 1000:
#             count += 1
# print(max_len)
# print(count)

1000
0


# 模型训练

In [4]:
% cd /content/gdrive/My\ Drive/Colab\ Notebooks/nlp_task/fine_gained/mycode
! pwd
! ls

/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
data		 elmo_no_all_drop.py  output	       run_log.ipynb
data_not_aug	 elmo.py	      preprocess.sh    weight_drop_lstm.py
data_process.py  elmo_run.py	      __pycache__
dataset.py	 elmo_utils.py	      run_log_@.ipynb


In [ ]:
# 裁剪前后文字的数据，效果在长度减小后变差
!python elmo_run.py \
--mode=train \
--data_files=data/clip_trian.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=32 \
--max_len=1000 \
--rnn_cell_name=gru \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=2000 \
--optimizer='adam' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0.1 \
--checkpoint_dir=output/elmo_ema_smoothing_aug

In [ ]:
# 未裁剪，只截断
!python elmo_run.py \
--mode=train \
--data_files=data/train.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1200 \
--rnn_cell_name='WEIGHT_LSTM' \
--linear_dropout=0.5 \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0 \
--checkpoint_dir=output/elmo_1200_more_drop_noSmooth

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# 原始数据
!python elmo_run.py \
--mode=train \
--data_files=data/train_not_aug.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1200 \
--rnn_cell_name='WEIGHT_LSTM' \
--linear_dropout=0.5 \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0 \
--checkpoint_dir=output/elmo_1200_more_drop_noSmooth_notAug \
--checkpoint_load_step=4000

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# fix embedding + 原始数据
!python elmo_run.py \
--mode=train \
--data_files=data/train_not_aug.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1400 \
--rnn_cell_name='WEIGHT_LSTM' \
--linear_dropout=0.5 \
--fix_embedding=True \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0 \
--checkpoint_dir=output/elmo_1200_more_drop_noSmooth_notAug_fix \
--checkpoint_load_step=11000 \
--previous_best_eval=-0.7024667962528659

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# softmax global loss
!python elmo_run.py \
--mode=train \
--data_files=data/train_not_aug.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1200 \
--rnn_cell_name='WEIGHT_LSTM' \
--linear_dropout=0.5 \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0 \
--checkpoint_dir=output/elmo_origin_loss

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# softmax global loss + fix embedding
!python elmo_run.py \
--mode=train \
--data_files=data/train_not_aug.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1200 \
--rnn_cell_name='WEIGHT_LSTM' \
--fix_embedding=True \
--linear_dropout=0.5 \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0 \
--checkpoint_dir=output/elmo_origin_loss_fix \
--checkpoint_load_step=12000 \
--previous_best_eval=-0.7055756348852982

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# softmax global loss + fix embedding: at 14000
!python elmo_run.py \
--mode=train \
--data_files=data/train_not_aug.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1200 \
--rnn_cell_name='WEIGHT_LSTM' \
--fix_embedding=True \
--linear_dropout=0.5 \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0 \
--checkpoint_dir=output/elmo_origin_loss_fix_at_best \
--checkpoint_load_step=14000 \
--previous_best_eval=-0.7075728796925465

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
!python elmo_run.py \
--mode=inference \
--data_files=data/testa.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--out_file=output/out.testa.json \
--prob=False \
--batch_size=100 \
--num_classes_each_label 4 \
--num_labels=20 \
--checkpoint_dir=output/elmo_ema_smoothing_aug

# 效果测试

In [ ]:
% cd /content/gdrive/My\ Drive/Colab\ Notebooks/nlp_task/fine_gained/mycode
! pwd
! ls

In [11]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 忽略警告
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
try:
    from tensorflow.python.util import module_wrapper as deprecation
except ImportError:
    from tensorflow.python.util import deprecation_wrapper as deprecation
deprecation._PER_MODULE_WARNING_LIMIT = 0

In [32]:
import tensorflow as tf
from elmo import Model
from dataset import DataSet
from collections import defaultdict
from elmo_utils import *


checkpoint_dir = "output/elmo_origin_loss_fix_at_best"


eval_dataset = DataSet(["data/validation.json"],
                           "data/vocab.txt",
                           "data/labels.txt",
                           128,
                           reverse=False,
                           split_word=True,
                           max_len=1200)

eval_graph = tf.Graph()

with eval_graph.as_default():
    eval_config = load_config(checkpoint_dir,
                                {"mode":'eval','checkpoint_dir':checkpoint_dir+"/best_eval"})
    eval_model = Model(eval_config)
    eval_model.build()

# vocab size:  50000
# vocab size:  20
# Start to preprocessing data...
# load data from data/validation.json ...
# Got 15000 data items with 117 batches
loading config from output/elmo_origin_loss_fix_at_best/config
# Start to load pretrained embedding...
# vocab size:  50000
# pretrained embedding size 44512 300
构建elmo静态图...


In [33]:
def cal_f1_and_more(class_num, predicted, truth):
    "class_num -- 4; predicted -- [15000, 4]; truth -- [15000, 4]"
    results = []
    for i in range(class_num):
        results.append({"TP": 0, "FP": 0, "FN": 0, "TN": 0})

    for i, p in enumerate(predicted):
        t = truth[i]
        for j in range(class_num):
            if p[j] == 1:
                if t[j] == 1:
                    results[j]['TP'] += 1
                else:
                    results[j]['FP'] += 1
            else:
                if t[j] == 1:
                    results[j]['FN'] += 1
                else:
                    results[j]['TN'] += 1

    precision = [0.0] * class_num
    recall = [0.0] * class_num
    f1 = [0.0] * class_num
    for i in range(class_num):
        if results[i]['TP'] == 0:
            if results[i]['FP'] == 0 and results[i]['FN'] == 0:
                precision[i] = 1.0
                recall[i] = 1.0
                f1[i] = 1.0
            else:
                precision[i] = 0.0
                recall[i] = 0.0
                f1[i] = 0.0
        else:
            precision[i] = results[i]['TP'] / (results[i]['TP'] +
                                               results[i]['FP'])
            recall[i] = results[i]['TP'] / (results[i]['TP'] +
                                            results[i]['FN'])
            f1[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])

    return f1, precision, recall, results

In [34]:
with tf.Session(graph=eval_graph, config=get_config_proto(log_device_placement=False)) as eval_sess:
    eval_model.init_model(eval_sess)
    eval_model.restore_model(eval_sess)

    checkpoint_loss, acc = 0.0, 0.0
    predicts, truths = defaultdict(list), defaultdict(list)
    for idx,(source, lengths, targets, _) in enumerate(eval_dataset.get_next(shuffle=False)):
        batch_loss, accuracy, batch_size, predict, logits = eval_model.eval_clf_one_step(eval_sess,
                                                                            source,
                                                                            lengths,
                                                                            targets)
        # predict： batch * 20 * 4
        for i, p in enumerate(predict):
            for j in range(eval_model.config.num_labels):
                label_name = eval_dataset.i2l[j]
                truths[label_name].append(targets[i][j])
                predicts[label_name].append(p[j])
        checkpoint_loss += batch_loss
        acc += accuracy
        if (idx+1) % 100 == 0:
            print("=>> batch %d/%d" %(i+1,eval_dataset.num_batches))

    results = {}
    total_f1 = 0.0
    metrics = {}
    for label_name in eval_dataset.label_names:
        f1, precision, recall, results_detail = cal_f1_and_more(eval_model.config.num_classes_each_label,
                                        np.asarray(predicts[label_name]),
                                        np.asarray(truths[label_name]))
        metrics[label_name] = {'f1': f1, 'precision': precision, 'recall': recall, 'details': results_detail}

        results[label_name] = sum(f1) / len(f1)
        total_f1 += sum(f1) / len(f1)
        print("=>> {0} - {1}".format(label_name, sum(f1) / len(f1)))

    final_f1 = total_f1 / len(results)

    print("=>> Eval loss %.5f, f1 %.5f" % (checkpoint_loss / i, final_f1))


!!! Restored model
=>> batch 128/117
=>> location_traffic_convenience - 0.6635102376478403
=>> location_distance_from_business_district - 0.5557976118306553
=>> location_easy_to_find - 0.7079569694634317
=>> service_wait_time - 0.6655879854095182
=>> service_waiters_attitude - 0.8004459100756012
=>> service_parking_convenience - 0.7401540201622667
=>> service_serving_speed - 0.7545546848035559
=>> price_level - 0.7839641368574368
=>> price_cost_effective - 0.7114510835772092
=>> price_discount - 0.6700410389334504
=>> environment_decoration - 0.7316427629299174
=>> environment_noise - 0.7646429859399563
=>> environment_space - 0.7706049828527306
=>> environment_cleaness - 0.7613696422917533
=>> dish_portion - 0.7274114163940149
=>> dish_taste - 0.7318627889496034
=>> dish_look - 0.5813661673335402
=>> dish_recommendation - 0.7345278066715328
=>> others_overall_experience - 0.59359095187412
=>> others_willing_to_consume_again - 0.7120806764505359
=>> Eval loss 1.71949, f1 0.70813


In [36]:
metrics

{'dish_look': {'details': [{'FN': 1189, 'FP': 755, 'TN': 11119, 'TP': 1937},
   {'FN': 515, 'FP': 168, 'TN': 14171, 'TP': 146},
   {'FN': 267, 'FP': 150, 'TN': 14395, 'TP': 188},
   {'FN': 829, 'FP': 1727, 'TN': 2515, 'TP': 9929}],
  'f1': [0.6658645582674458,
   0.29948717948717946,
   0.4741488020176544,
   0.885964129561881],
  'precision': [0.7195393759286776,
   0.46496815286624205,
   0.5562130177514792,
   0.8518359643102265],
  'recall': [0.6196417146513116,
   0.22087745839636913,
   0.41318681318681316,
   0.9229410671128463]},
 'dish_portion': {'details': [{'FN': 830, 'FP': 935, 'TN': 10024, 'TP': 3211},
   {'FN': 707, 'FP': 487, 'TN': 13108, 'TP': 698},
   {'FN': 410, 'FP': 415, 'TN': 13143, 'TP': 1032},
   {'FN': 992, 'FP': 1102, 'TN': 5786, 'TP': 7120}],
  'f1': [0.7844143153780385,
   0.5389961389961389,
   0.7144340602284527,
   0.8718011509734296],
  'precision': [0.7744814278822962,
   0.5890295358649789,
   0.7131997235659986,
   0.8659693505229871],
  'recall': [0.7

In [27]:
import pickle

with open("data/class_weights.bin", 'rb') as f:
    class_weight = pickle.load(f, encoding='utf-8')

In [37]:
class_weight_inverse = [[1 / num for num in list_a] for list_a in class_weight]

In [ ]:
class_weight_inverse

In [44]:
f1_list = []
precision_list = []
recall_list = []

for i, label_name in enumerate(eval_dataset.label_names):
    weighted_f1 = sum([a * b for a, b in zip(class_weight_inverse[i], metrics[label_name]['f1'])])
    weighted_precision = sum([a * b for a, b in zip(class_weight_inverse[i], metrics[label_name]['precision'])])
    weighted_recall = sum([a * b for a, b in zip(class_weight_inverse[i], metrics[label_name]['recall'])])
    print("=" * 30)
    print(label_name)
    print("weighted f1:", weighted_f1)
    print("weighted precision:", weighted_precision)
    print("weighted recall:", weighted_recall)
    f1_list.append(weighted_f1)
    precision_list.append(weighted_precision)
    recall_list.append(weighted_recall)

print("#" * 30)
print("averaged f1:", sum(f1_list) / len(f1_list))
print("averaged precision:", sum(precision_list) / len(precision_list))
print("averaged recall:", sum(recall_list) / len(recall_list))

location_traffic_convenience
weighted f1: 0.9455280035707135
weighted precision: 0.9445345707317678
weighted recall: 0.9482195540666717
location_distance_from_business_district
weighted f1: 0.8881375561561486
weighted precision: 0.8870191135408428
weighted recall: 0.8904373110744659
location_easy_to_find
weighted f1: 0.9183189314084468
weighted precision: 0.9166392496604572
weighted recall: 0.9225109936312154
service_wait_time
weighted f1: 0.9164206380868779
weighted precision: 0.9151522657408971
weighted recall: 0.917984210073227
service_waiters_attitude
weighted f1: 0.8682723844621453
weighted precision: 0.866427990492425
weighted recall: 0.8717010343757985
service_parking_convenience
weighted f1: 0.9766218809945754
weighted precision: 0.9760819518426841
weighted recall: 0.97791685413215
service_serving_speed
weighted f1: 0.9398847749451685
weighted precision: 0.9389127136183217
weighted recall: 0.9416639639190226
price_level
weighted f1: 0.818927262571223
weighted precision: 0.82223

# 结论

单模型最高macro f1达到 70.81%。多模型集成会有提高。

数据极度不平衡。计算weighted平均precision 最好达到0.8746，weighted平均f1最高达到0.8744，weighted平均recall最高达到0.8770。

效果相比于基本的Albert classification有很大的提升。相比于多个单模型集成的fasttext，结果有更好。

基本的Albert classification一是难以完成长文本的信息的完整获取，而是模型在少量fine tuning的基础之上，拼接上层更复杂的模型，以提升拟合能力可能会有效果的提升。